# Scalogram UI

In [1]:
import wfdb
import matplotlib as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import resample
import numpy as np
import neurokit2 as nk
import pywt
import seaborn as sns
import os


In [2]:
# 현재 경로 불러오기 = os.getcwd()
path = os.getcwd()
os.chdir(path+ "\\training2017")
print(os.getcwd())


d:\Code Works\Work\vscode\ecgproject\Spectrogram\training2017


In [3]:
csv='REFERENCE.csv'
data = pd.read_csv(csv, header=None, names=['ID', 'Label'])

In [4]:
def ecg_read(patient_id):
    record = wfdb.rdrecord(patient_id) 
    ecg_signal = record.p_signal[:,0]  
    fs = record.fs
    return ecg_signal, fs

In [5]:
from scipy.signal import butter, filtfilt
class Pan_tompkins:
    """ Implementationof Pan Tompkins Algorithm.

    Noise cancellation (bandpass filter) -> Derivative step -> Squaring and integration.

    Params:
        data (array) : ECG data
        sampling rate (int)
    returns:
        Integrated signal (array) : This signal can be used to detect peaks


    ----------------------------------------
    HOW TO USE ?
    Eg.

    ECG_data = [4, 7, 80, 78, 9], sampling  =2000
    
    call : 
       signal = Pan_tompkins(ECG_data, sampling).fit()

    ----------------------------------------
    
    """
    def __init__(self, data, sample_rate):

        self.data = data
        self.sample_rate = sample_rate


    def fit(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150, window_size=None):
        ''' Fit the signal according to algorithm and returns integrated signal
        
        '''
        # 1.Noise cancellationusing bandpass filter
        self.filtered_BandPass = self.band_pass_filter(normalized_cut_offs, butter_filter_order, padlen)
        
        # 2.derivate filter to get slpor of the QRS
        self.derviate_pass = self.derivative_filter()

        # 3.Squaring to enhance dominant peaks in QRS
        self.square_pass = self.squaring()

        # 4.To get info about QRS complex
        self.integrated_signal = self.moving_window_integration( window_size)

        return self.integrated_signal


    def band_pass_filter(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150):
        ''' Band pass filter for Pan tompkins algorithm
            with a bandpass setting of 5 to 20 Hz

            params:
                normalized_cut_offs (list) : bandpass setting canbe changed here
                bandpass filte rorder (int) : deffault 2
                padlen (int) : padding length for data , default = 150
                        scipy default value = 2 * max(len(a coeff, b coeff))

            return:
                filtered_BandPass (array)
        '''

        # Calculate nyquist sample rate and cutoffs
        nyquist_sample_rate = self.sample_rate / 2

        # calculate cutoffs
        if normalized_cut_offs is None:
            normalized_cut_offs = [5/nyquist_sample_rate, 15/nyquist_sample_rate]
        else:
            assert type(self.sample_rate ) is list, "Cutoffs should be a list with [low, high] values"

        # butter coefficinets 
        b_coeff, a_coeff = butter(butter_filter_order, normalized_cut_offs, btype='bandpass')[:2]

        # apply forward and backward filter
        filtered_BandPass = filtfilt(b_coeff, a_coeff, self.data, padlen=padlen)
        
        return filtered_BandPass


    def derivative_filter(self):
        ''' Derivative filter

        params:
            filtered_BandPass (array) : outputof bandpass filter
        return:
            derivative_pass (array)
        '''

        # apply differentiation
        derviate_pass= np.diff(self.band_pass_filter())

       

        return derviate_pass


    def squaring(self):
        ''' squaring application on derivate filter output data

        params:

        return:
            square_pass (array)
        '''

        # apply squaring
        square_pass= self.derivative_filter() **2

        return square_pass 


    def moving_window_integration(self, window_size=None):
        ''' Moving avergae filter 

        Params:
            window_size (int) : no. of samples to average, if not provided : 0.08 * sample rate
            sample_rate (int) : should be given if window_size is not given  
        return:
            integrated_signal (array)
        '''

        if window_size is None:
            assert self.sample_rate is not None, "if window size is None, sampling rate should be given"
            window_size = int(0.08 * int(self.sample_rate))  # given in paper 150ms as a window size
        

        # define integrated signal
        integrated_signal = np.zeros_like(self.squaring())

        # cumulative sum of signal
        cumulative_sum = self.squaring().cumsum()

        # estimationof area/ integral below the curve deifnes the data
        integrated_signal[window_size:] = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size

        integrated_signal[:window_size] = cumulative_sum[:window_size] / np.arange(1, window_size + 1)

        return integrated_signal

In [6]:
A =[] # Atrial Fibrillation
N= [] # Normal Sinus rhythm
O = [] # Other rhythm
I = [] # Noisy

for id, lb in zip (data['ID'], data['Label'] ): #v2 data
         
         if lb =='A':
            A.append(id)
            
         elif lb =='N':
            N.append(id)
         
         elif lb =='O':
            O.append(id)
        
         elif lb =='~':
            I.append(id)   

In [7]:
dic = {'Atrial Fibrillation': A, 'Normal Sinus Rhythm': N, 'Other Rhythm': O, 'Noisy': I}

In [8]:
# Resample ECG signal from original sampling rate to target sampling rate -> 해상도
def resample_ecg_signal(ecg_signal, original_fs, target_fs):
    num_samples = int(len(ecg_signal) * target_fs / original_fs)
    resampled_signal = resample(ecg_signal, num_samples)
    return resampled_signal

> none db

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import wfdb
from ipywidgets import interact, interactive, IntSlider, FloatSlider, Dropdown, fixed, HBox, Layout, Checkbox
from IPython.display import display, clear_output
import scipy.signal as signal
import neurokit2 as nk

# Define the wavelet
wavelet = 'morl'

# ECG 데이터 로드 함수 (가정)
def ecg_read(record_name):
    record = wfdb.rdrecord(record_name)
    ecg_signal = record.p_signal[:, 0]
    fs = record.fs
    return ecg_signal, fs

def normalize_ecg(ecg_signal):
    min_val = np.min(ecg_signal)
    max_val = np.max(ecg_signal)
    normalized_signal = (ecg_signal - min_val) / (max_val - min_val)
    return normalized_signal 

def normalize_wavelet_coeffs(wavelet_coeffs):
    min_val = np.min(wavelet_coeffs)
    max_val = np.max(wavelet_coeffs)
    normalized_wavelet_coeffs= 2*((wavelet_coeffs - min_val) / (max_val - min_val))-1  # 1~-1
    normalized_wavelet_coeffs= 10*(2*((wavelet_coeffs - min_val) / (max_val - min_val))-1)  #10~-10
    return normalized_wavelet_coeffs
 

def plot_segmented_beats(ecg_signal, fs, start_index, end_index, padding_length_s, tukey_apply, alpha, reverse, max_scale, pantompkins, wavelet, Nomalization, colormap, apply_vmin_vmax, vmax,vmin):
       
    if pantompkins:
        ecg_signal = Pan_tompkins(ecg_signal, fs).fit()

    if Nomalization:
        ecg_signal = normalize_ecg(ecg_signal)

    if padding_length_s == 0:
        segment = ecg_signal[start_index:end_index]
        padding_length = 0
    else:    
        segment = ecg_signal[start_index:end_index]
        padding_length = int(fs / padding_length_s)
        segment = np.pad(segment, (padding_length, padding_length), mode='reflect')

    if tukey_apply:
        tukey_window = signal.windows.tukey(len(segment), alpha=alpha)
        segment_padded = segment * tukey_window
    else:
        segment_padded = segment

    scales = np.arange(1, max_scale + 1)
    wavelet_coeffs, freqs = pywt.cwt(segment_padded, scales, wavelet=wavelet, sampling_period=1 / fs)

    # wavelet_coeffs=normalize_wavelet_coeffs(wavelet_coeffs)

    # Calculate the time axis 
    times = np.linspace(0, len(segment_padded) / fs, len(segment_padded))

    # Set up the plot for the ECG segment and the scalogram
    plt.figure(figsize=(15, 2))

    # Plot the ECG segment
    plt.subplot(1, 2, 1)
    plt.plot(times, segment_padded, label='ECG Signal')
    plt.title(f'ECG Segment ({start_index / fs:.2f}s to {end_index / fs:.2f}s) +{2 * (padding_length / fs):.2f}s')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()

    # Plot the scalogram
    plt.subplot(1, 2, 2)
    extent = [0, len(segment_padded) / fs, 1, len(scales)]
    if apply_vmin_vmax:
        plt.imshow(np.abs(wavelet_coeffs), cmap=colormap, aspect='auto', extent=extent, vmin=vmin, vmax=vmax)
    else:
        plt.imshow(np.abs(wavelet_coeffs), cmap=colormap, aspect='auto', extent=extent)
    plt.title('Scalogram')
    plt.xlabel('Time')
    plt.ylabel('Scales')
    plt.colorbar()

    plt.show()

    print(f'Vmax: {np.max(wavelet_coeffs)}')
    print(f'Vmin: {np.min(wavelet_coeffs)}')

    # Plot the polar scalogram
    theta = 2 * np.pi * times / max(times)
    rho = np.linspace(0.1, max(scales), len(scales))
    T, R = np.meshgrid(theta, rho)

    plt.figure(figsize=(5, 5))
    ax = plt.subplot(1, 1, 1, polar=True)
    if reverse:
        if apply_vmin_vmax: 
            c = ax.pcolormesh(T, R, np.flipud(np.abs(wavelet_coeffs)), shading='auto', cmap=colormap, vmin=vmin, vmax=vmax)
        else:
            c = ax.pcolormesh(T, R, np.flipud(np.abs(wavelet_coeffs)), shading='auto', cmap=colormap)     
    else:
        if apply_vmin_vmax: 
            c = ax.pcolormesh(T, R, np.abs(wavelet_coeffs), shading='auto', cmap=colormap, vmin=vmin, vmax=vmax)
        else:
            c = ax.pcolormesh(T, R, np.abs(wavelet_coeffs), shading='auto', cmap=colormap)     
            
    plt.title('Polar Scalogram')

    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)

    num_time_points = 12
    time_labels = [f'{t:.1f}s' for t in np.linspace(0, max(times), num_time_points)]
    theta_ticks = np.linspace(0, 2 * np.pi, num_time_points, endpoint=False)
    ax.set_xticks(theta_ticks)
    ax.set_xticklabels(time_labels)

    ax.set_rgrids([])
    ax.grid(alpha=0)
    plt.tight_layout()
    plt.show()

# Initialize dropdowns
label_dropdown = Dropdown(options=list(dic.keys()), description='ECG Label:', layout=Layout(width='50%'))
patient_dropdown = Dropdown(description='Patient ID:', options=[], layout=Layout(width='50%'))
reverse_dropdown = Dropdown(options=[True, False], value=True, description='Reverse')
wavelet_dropdown = Dropdown(options=list(pywt.wavelist(kind='continuous')), value='mexh', description='Wavelet')
pantompkins_dropdown = Dropdown(options=[True, False], value=True, description='Pantompkins')
Normalization_dropdown = Dropdown(options=[True, False], value=False, description='Normalization')

# Tukey window 선택 여부와 alpha 슬라이더
tukey_apply_dropdown = Dropdown(options=[True, False], value=True, description='Apply Tukey')
alpha_slider = FloatSlider(min=0, max=1, step=0.01, value=0.05, description='Tukey Alpha:', continuous_update=False)

# Generate list of powers of 2 for the scale slider
scale_values = [2 ** i for i in range(1, 11)]
scale_dropdown = Dropdown(options=scale_values, value=128, description='Max Scale:', layout=Layout(width='50%'))

# Colormap dropdown
colormap_dropdown = Dropdown(options=['jet', 'turbo', 'viridis', 'plasma', 'inferno', 'magma', 'cividis', 'flag', 'prism', 'ocean', 'gist_earth', 'terrain',
                      'gist_stern', 'gnuplot', 'gnuplot2', 'CMRmap',
                      'cubehelix', 'brg', 'gist_rainbow', 'rainbow', 'tab10','tab20b', 'tab20c',
                       'nipy_spectral', 'gist_ncar'], value='jet', description='Colormap')

# Vmin and Vmax sliders
apply_vmin_vmax_dropdown = Dropdown(options=[True, False], value=False, description='Apply Vmin/Vmax')
vmax_slider = FloatSlider(min=0, max=1, step=0.01, value= 0.01355, description='Vmax:')
vmin_slider = FloatSlider(min=-1, max=1, step=0.01, value=-0.008514, description='Vmin:')



# Tukey 적용 여부에 따라 alpha 슬라이더 활성화/비활성화
def update_alpha_slider(*args):
    alpha_slider.disabled = not tukey_apply_dropdown.value

tukey_apply_dropdown.observe(update_alpha_slider, names='value')

def update_patient_options(*args):
    patient_dropdown.options = dic[label_dropdown.value]

label_dropdown.observe(update_patient_options, 'value')

def interactive_plot(patient_id):
    ecg_signal, fs = ecg_read(patient_id)
    max_index = len(ecg_signal)
    start_slider = IntSlider(min=0, max=max_index, step=int(0.5 * fs), value=0, description='Start Index:')
    end_slider = IntSlider(min=0, max=max_index, step=int(0.5 * fs), value=9000, description='End Index:')
    padding_slider = FloatSlider(min=0, max=10, step=0.1, value=0, description='Pad len(s):')

    return interactive(plot_segmented_beats, ecg_signal=fixed(ecg_signal), fs=fixed(fs),
                       start_index=start_slider, end_index=end_slider, padding_length_s=padding_slider,
                       tukey_apply=tukey_apply_dropdown, alpha=alpha_slider, reverse=reverse_dropdown,
                       max_scale=scale_dropdown, pantompkins=pantompkins_dropdown, wavelet=wavelet_dropdown,
                       Nomalization=Normalization_dropdown, colormap=colormap_dropdown, 
                       apply_vmin_vmax=apply_vmin_vmax_dropdown,vmax=vmax_slider, vmin=vmin_slider,)

# Connects the plotting function to the selected patient ID
def on_patient_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(HBox([label_dropdown, patient_dropdown]))
        plot_ui = interactive_plot(change['new'])
        display(HBox([plot_ui]))  # Added vmin/vmax toggle dropdown
        
patient_dropdown.observe(on_patient_selected, names='value')

display(HBox([label_dropdown, patient_dropdown]))





In [10]:
# from scipy.signal import resample

# # Resample ECG signal from original sampling rate to target sampling rate -> 해상도
# def resample_ecg_signal(ecg_signal, original_fs, target_fs):
#     num_samples = int(len(ecg_signal) * target_fs / original_fs)
#     resampled_signal = resample(ecg_signal, num_samples)
#     return resampled_signal

# # Main execution
# record_path = 'A00001'
# ecg_signal, fs = ecg_read(record_path)
# # ecg_signal = resample_ecg_signal(ecg_signal, fs, 50)
# # fs=50

# # ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
# # signals, info = nk.ecg_process(ecg_filtered , sampling_rate=fs,method='pantompkins1985')
# # ecg_signal = signals["ECG_Clean"]

# # Parameters
# padding_length_s = 8
# max_scale = 124
# alpha=0.05
# wavelet = 'mexh'

# segment= ecg_signal
# # Calculate padding length in samples
# padding_length = int(fs / padding_length_s)  # Since fs is in samples per second

# # Apply zero-padding at the beginning and end of the segment
# ecg_signal_padded = np.pad(segment, (padding_length, padding_length), 'constant', constant_values=(0, 0))

# # Apply Tukey window to the padded signal
# tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=0.05)
# ecg_signal_padded *= tukey_window

# # Perform continuous wavelet transform (CWT)
# scales = np.arange(1, max_scale + 1)
# wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

# # Calculate the time axis 
# times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))

# # Generate the polar spectrogram
# theta = 2 * np.pi * times_padded / max(times_padded)
# rho = np.linspace(0.1, max_scale, max_scale)
# T, R = np.meshgrid(theta, rho)

# polar_data = np.flipud(np.abs(wavelet_coeffs+1e-10))


# image_size=(64,64)
# dpi=100
# cmap='turbo'
# vmin=0
# vmax=1

# fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
# fig, ax = plt.subplots(figsize=(5,5), dpi=dpi, subplot_kw={'projection': 'polar'})
# # fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi, subplot_kw={'projection': 'polar'})
# c = ax.pcolormesh(T, R, polar_data, shading='auto', cmap=cmap, vmin=vmin, vmax=vmax)
# # c = ax.pcolormesh(T, R, polar_data, shading='auto', cmap=cmap, )

# # Remove axis lines, labels, and ticks
# ax.set_theta_zero_location('N')  # 0 degrees at the top
# ax.set_theta_direction(-1)  # Clockwise direction
# ax.axis('off')

# # Ensure there is no padding and no border around the image
# plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
# ax.margins(0, 0)
# ax.xaxis.set_major_locator(plt.NullLocator())
# ax.yaxis.set_major_locator(plt.NullLocator())

# plt.show()

> 해상도가 안나오는 이유
- 1. wavelet 특징 (morlet이 이미지 분류에 부족)
- 2. ecg 해상도?

In [11]:
# pywt.wavelist(kind='continuous')

## 보간법

In [12]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pywt
# from scipy.signal import resample
# import cv2
# from scipy.ndimage import zoom

# # Resample ECG signal from original sampling rate to target sampling rate -> 해상도
# def resample_ecg_signal(ecg_signal, original_fs, target_fs):
#     num_samples = int(len(ecg_signal) * target_fs / original_fs)
#     resampled_signal = resample(ecg_signal, num_samples)
#     return resampled_signal


# # 시각화
# def plot_interpolation_result(data, title, image_size):
#     fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
#     fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi, subplot_kw={'projection': 'polar'})
#     # fig, ax = plt.subplots(figsize=(5,5), dpi=dpi, subplot_kw={'projection': 'polar'})
#     c = ax.pcolormesh(T, R, data, shading='auto', cmap=cmap)
#     ax.set_theta_zero_location('N')
#     ax.set_theta_direction(-1)
#     ax.axis('off')
#     plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
#     ax.margins(0, 0)
#     ax.xaxis.set_major_locator(plt.NullLocator())
#     ax.yaxis.set_major_locator(plt.NullLocator())
#     plt.title(title)
#     plt.show()


# # Main execution
# record_path = 'A00004'
# ecg_signal, fs = ecg_read(record_path)

# ecg_signal = resample_ecg_signal(ecg_signal, fs, 500)
# fs = 500

# ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
# signals, info = nk.ecg_process(ecg_filtered , sampling_rate=fs,method='pantompkins1985')
# ecg_signal = signals["ECG_Clean"]

# # Parameters
# padding_length_s = 8
# max_scale = 128
# alpha = 0.05
# wavelet = 'morl'  # ['cgau1','cgau2','cgau3','cgau4','cgau5','cgau6','cgau7','cgau8','cmor1.5-1.0','fbsp1-1.5-1.0','gaus1','gaus2','gaus3','gaus4','gaus5','gaus6','gaus7','gaus8','mexh','morl','shan1.5-1.0']

# ecg_signal_padded = ecg_signal
# # Calculate padding length in samples
# padding_length = int(fs / padding_length_s)  # Since fs is in samples per second

# # Apply zero-padding at the beginning and end of the segment
# ecg_signal_padded = np.pad(segment, (padding_length, padding_length), 'constant', constant_values=(0, 0))

# # Apply Tukey window to the padded signal
# tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=alpha)
# ecg_signal_padded = ecg_signal_padded * tukey_window

# # Perform continuous wavelet transform (CWT)
# scales = np.arange(1, max_scale + 1)
# wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

# # Calculate the time axis 
# times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))

# # Generate the polar spectrogram
# theta = 2 * np.pi * times_padded / max(times_padded)
# rho = np.linspace(0.1, max_scale, max_scale)
# T, R = np.meshgrid(theta, rho)

# polar_data = np.flipud(np.abs(wavelet_coeffs + 1e-10))

# plot_interpolation_result(polar_data,'Original Image',image_size=image_size)

# # Interpolation
# image_size =(128, 128)  # 업스케일링된 이미지 크기
# dpi = 100
# cmap = 'turbo'
# # 기존 이미지


# # Resize using nearest-neighbor interpolation
# nearest_neighbor = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_NEAREST)

# # Resize using bilinear interpolation
# bilinear = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_LINEAR)

# # Resize using bicubic interpolation
# bicubic = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_CUBIC)

# # Resize using spline interpolation (order=3 for cubic spline)
# spline = zoom(polar_data, zoom=(image_size[0] / polar_data.shape[0], image_size[1] / polar_data.shape[1]), order=3)

# # Adjust the meshgrid dimensions to fit the new data
# T, R = np.meshgrid(np.linspace(theta.min(), theta.max(), nearest_neighbor.shape[1]),
#                    np.linspace(rho.min(), rho.max(), nearest_neighbor.shape[0]))


# # 각 보간 방법에 따른 결과 시각화

# plot_interpolation_result(nearest_neighbor, 'Nearest-Neighbor Interpolation',image_size=image_size)
# plot_interpolation_result(bilinear, 'Bilinear Interpolation',image_size=image_size)
# plot_interpolation_result(bicubic, 'Bicubic Interpolation',image_size=image_size)
# plot_interpolation_result(spline, 'Spline Interpolation',image_size=image_size)


## ECG Normalization

In [13]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 최소-최대 정규화 함수
# def normalize_ecg(ecg_signal):
#     min_val = np.min(ecg_signal)
#     max_val = np.max(ecg_signal)
#     normalized_signal = (ecg_signal - min_val) / (max_val - min_val)
#     return normalized_signal


# # 시각화
# def plot_interpolation_result(data, title, image_size):
#     fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
#     # fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi, subplot_kw={'projection': 'polar'})
#     fig, ax = plt.subplots(figsize=(5,5), dpi=dpi, subplot_kw={'projection': 'polar'})
#     c = ax.pcolormesh(T, R, data, shading='auto', cmap=cmap)
#     ax.set_theta_zero_location('N')
#     ax.set_theta_direction(-1)
#     ax.axis('off')
#     plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
#     ax.margins(0, 0)
#     ax.xaxis.set_major_locator(plt.NullLocator())
#     ax.yaxis.set_major_locator(plt.NullLocator())
#     plt.title(title)
#     plt.show()

# # ECG 신호 불러오기
# record_path = 'A00001'
# ecg_signal, fs = ecg_read(record_path)
# times = np.linspace(0, len(ecg_signal) / fs, len(ecg_signal))

# # ECG 신호 정규화
# normalized_ecg_signal = normalize_ecg(ecg_signal)

# ecg_signal=ecg_signal[:1200]

# # 정규화된 신호 시각화
# plt.figure(figsize=(12, 3))
# plt.plot(ecg_signal, label='ECG Signal', color='green')
# plt.title(f'ECG Signal')
# plt.xlabel('Time (s)')
# plt.ylabel('Amplitude')
# plt.legend()
# plt.show()

# # # 정규화된 신호 시각화
# # plt.figure(figsize=(12, 3))
# # plt.plot(times, normalized_ecg_signal, label='Normalized ECG Signal', color='r')
# # plt.title(f'Normalized ECG Signal')
# # plt.xlabel('Time (s)')
# # plt.ylabel('Amplitude')
# # plt.legend()
# # plt.show()

# ecg_signal=normalized_ecg_signal
# # ecg_signal = resample_ecg_signal(ecg_signal, fs, 500)
# # fs = 500

# # ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
# # signals, info = nk.ecg_process(ecg_filtered , sampling_rate=fs,method='pantompkins1985')
# # ecg_signal = signals["ECG_Clean"]

# # # Parameters
# padding_length_s = 8
# max_scale = 128
# alpha = 0.05
# wavelet = 'mexh'  # ['cgau1','cgau2','cgau3','cgau4','cgau5','cgau6','cgau7','cgau8','cmor1.5-1.0','fbsp1-1.5-1.0','gaus1','gaus2','gaus3','gaus4','gaus5','gaus6','gaus7','gaus8','mexh','morl','shan1.5-1.0']

# ecg_signal_padded = ecg_signal
# # Calculate padding length in samples
# padding_length = int(fs / padding_length_s)  # Since fs is in samples per second

# # Apply zero-padding at the beginning and end of the segment
# ecg_signal_padded = np.pad(ecg_signal_padded, (padding_length, padding_length), 'constant', constant_values=(0, 0))

# # Apply Tukey window to the padded signal
# tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=alpha)
# ecg_signal_padded = ecg_signal_padded * tukey_window

# # Perform continuous wavelet transform (CWT)
# scales = np.arange(1, max_scale + 1)
# wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

# # Calculate the time axis 
# times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))

# # Generate the polar spectrogram
# theta = 2 * np.pi * times_padded / max(times_padded)
# rho = np.linspace(0.1, max_scale, max_scale)
# T, R = np.meshgrid(theta, rho)

# polar_data = np.flipud(np.abs(wavelet_coeffs + 1e-10))

# # Interpolation
# image_size =(128, 128)  # 업스케일링된 이미지 크기
# dpi = 100
# cmap = 'turbo'


# plot_interpolation_result(polar_data,'Normalized Image',image_size=image_size)


# # # Resize using nearest-neighbor interpolation
# # nearest_neighbor = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_NEAREST)

# # # Resize using bilinear interpolation
# # bilinear = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_LINEAR)

# # # Resize using bicubic interpolation
# # bicubic = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_CUBIC)

# # # Resize using spline interpolation (order=3 for cubic spline)
# # spline = zoom(polar_data, zoom=(image_size[0] / polar_data.shape[0], image_size[1] / polar_data.shape[1]), order=3)

# # # Adjust the meshgrid dimensions to fit the new data
# # T, R = np.meshgrid(np.linspace(theta.min(), theta.max(), nearest_neighbor.shape[1]),
# #                    np.linspace(rho.min(), rho.max(), nearest_neighbor.shape[0]))


# # # 각 보간 방법에 따른 결과 시각화

# # plot_interpolation_result(nearest_neighbor, 'Nearest-Neighbor Interpolation',image_size=image_size)
# # plot_interpolation_result(bilinear, 'Bilinear Interpolation',image_size=image_size)
# # plot_interpolation_result(bicubic, 'Bicubic Interpolation',image_size=image_size)
# # plot_interpolation_result(spline, 'Spline Interpolation',image_size=image_size)



In [14]:
# import numpy as np
# import matplotlib.pyplot as plt

# # 최소-최대 정규화 함수
# def normalize_ecg(ecg_signal):
#     min_val = np.min(ecg_signal)
#     max_val = np.max(ecg_signal)
#     normalized_signal = (ecg_signal - min_val) / (max_val - min_val)
#     return normalized_signal


# # 시각화
# def plot_interpolation_result(data, title, image_size):
#     fig_size_inch = (image_size[0] / dpi, image_size[1] / dpi)
#     # fig, ax = plt.subplots(figsize=fig_size_inch, dpi=dpi, subplot_kw={'projection': 'polar'})
#     fig, ax = plt.subplots(figsize=(5,5), dpi=dpi, subplot_kw={'projection': 'polar'})
#     c = ax.pcolormesh(T, R, data, shading='auto', cmap=cmap)
#     ax.set_theta_zero_location('N')
#     ax.set_theta_direction(-1)
#     ax.axis('off')
#     plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
#     ax.margins(0, 0)
#     ax.xaxis.set_major_locator(plt.NullLocator())
#     ax.yaxis.set_major_locator(plt.NullLocator())
#     plt.title(title)
#     plt.show()

# # ECG 신호 불러오기
# record_path = 'A00001'
# ecg_signal, fs = ecg_read(record_path)
# times = np.linspace(0, len(ecg_signal) / fs, len(ecg_signal))

# # ECG 신호 정규화
# normalized_ecg_signal = normalize_ecg(ecg_signal)

# # 정규화된 신호 시각화
# # plt.figure(figsize=(12, 3))
# # plt.plot(times, ecg_signal, label='ECG Signal', color='green')
# # plt.title(f'ECG Signal')
# # plt.xlabel('Time (s)')
# # plt.ylabel('Amplitude')
# # plt.legend()
# # plt.show()

# # # 정규화된 신호 시각화
# # plt.figure(figsize=(12, 3))
# # plt.plot(times, normalized_ecg_signal, label='Normalized ECG Signal', color='r')
# # plt.title(f'Normalized ECG Signal')
# # plt.xlabel('Time (s)')
# # plt.ylabel('Amplitude')
# # plt.legend()
# # plt.show()

# # ecg_signal=normalized_ecg_signal
# # ecg_signal = resample_ecg_signal(ecg_signal, fs, 500)
# # fs = 500

# # ecg_filtered = nk.ecg_clean(ecg_signal, sampling_rate=fs, method="biosppy")
# # signals, info = nk.ecg_process(ecg_filtered , sampling_rate=fs,method='pantompkins1985')
# # ecg_signal = signals["ECG_Clean"]

# # # Parameters
# padding_length_s = 8
# max_scale = 128
# alpha = 0.05
# wavelet = 'mexh'  # ['cgau1','cgau2','cgau3','cgau4','cgau5','cgau6','cgau7','cgau8','cmor1.5-1.0','fbsp1-1.5-1.0','gaus1','gaus2','gaus3','gaus4','gaus5','gaus6','gaus7','gaus8','mexh','morl','shan1.5-1.0']

# ecg_signal_padded = ecg_signal
# # Calculate padding length in samples
# padding_length = int(fs / padding_length_s)  # Since fs is in samples per second

# # Apply zero-padding at the beginning and end of the segment
# ecg_signal_padded = np.pad(ecg_signal_padded, (padding_length, padding_length), 'constant', constant_values=(0, 0))

# # Apply Tukey window to the padded signal
# tukey_window = signal.windows.tukey(len(ecg_signal_padded), alpha=alpha)
# ecg_signal_padded = ecg_signal_padded * tukey_window

# # Perform continuous wavelet transform (CWT)
# scales = np.arange(1, max_scale + 1)
# wavelet_coeffs, freqs = pywt.cwt(ecg_signal_padded, scales, wavelet, sampling_period=1/fs)

# # Calculate the time axis 
# times_padded = np.linspace(0, len(ecg_signal_padded) / fs, len(ecg_signal_padded))

# # Generate the polar spectrogram
# theta = 2 * np.pi * times_padded / max(times_padded)
# rho = np.linspace(0.1, max_scale, max_scale)
# T, R = np.meshgrid(theta, rho)

# polar_data = np.flipud(np.abs(wavelet_coeffs + 1e-10))

# # Interpolation
# image_size =(128, 128)  # 업스케일링된 이미지 크기
# dpi = 100
# cmap = 'turbo'


# plot_interpolation_result(polar_data,'Original Image',image_size=image_size)


# # # Resize using nearest-neighbor interpolation
# # nearest_neighbor = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_NEAREST)

# # # Resize using bilinear interpolation
# # bilinear = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_LINEAR)

# # # Resize using bicubic interpolation
# # bicubic = cv2.resize(polar_data, image_size, interpolation=cv2.INTER_CUBIC)

# # # Resize using spline interpolation (order=3 for cubic spline)
# # spline = zoom(polar_data, zoom=(image_size[0] / polar_data.shape[0], image_size[1] / polar_data.shape[1]), order=3)

# # # Adjust the meshgrid dimensions to fit the new data
# # T, R = np.meshgrid(np.linspace(theta.min(), theta.max(), nearest_neighbor.shape[1]),
# #                    np.linspace(rho.min(), rho.max(), nearest_neighbor.shape[0]))


# # # 각 보간 방법에 따른 결과 시각화

# # plot_interpolation_result(nearest_neighbor, 'Nearest-Neighbor Interpolation',image_size=image_size)
# # plot_interpolation_result(bilinear, 'Bilinear Interpolation',image_size=image_size)
# # plot_interpolation_result(bicubic, 'Bicubic Interpolation',image_size=image_size)
# # plot_interpolation_result(spline, 'Spline Interpolation',image_size=image_size)

